In [ ]:
import modules.secret
from modules.Graph import Graph
import modules.FilterPassing as flt_pass
import os
import pprint
import pickle

pp = pprint.PrettyPrinter(indent=4, width=100)

SCORING_PATH = modules.secret.PRODUCTION_RUNS_PATH + "2. Generation/smiles/"
EXPORT_PATH = os.path.join(os.getcwd(), "exports", "admet_satisfaction", "")
SCORING_PATH

In [ ]:
# configs = [
#     ("model7_hnh_admet", "HNH", "ADMET", "softsub", 5, 0),
#     ("model7_hnh_admetfg", "HNH", "ADMET+FGs", "softsub", 5, 0),
#     ("model2_hnh", "HNH", "ADMET+FGs", "admetfg_softsub", 5, 0),
#     ("model7_1iep_admet", "1IEP", "ADMET", "softsub", 10, 0.02),
#     ("model7_1iep_admetfg", "1IEP", "ADMET+FGs", "softsub", 10, 0.02),
#     ("model2_1iep", "1IEP", "ADMET+FGs", "admetfg_softsub", 20, 0),
# ]

In [ ]:
prefix = "model2_hnh"
n_iters = 5
channel = "admetfg_softsub"
filters = "ADMET+FGs"
target = "HNH"
ignored = {"fChar"}
fnames = flt_pass.prepare_generation_fnames(
    prefix=prefix, n_iters=n_iters, channel=channel, filters=filters, target=target
)
load_generation = flt_pass.prepare_generation_loader(base_path=SCORING_PATH)
traces_lists = []
max_val = -float("inf")
for i, fname in enumerate(fnames):
    smiles = load_generation(fname)
    filtToData = flt_pass.compute_admet_metrics(smiles)
    traces, i_max_val = flt_pass.create_admet_metrics_traces(
        filtToData, showlegend=i == 0, ignored_metrics=ignored
    )
    max_val = max(max_val, i_max_val)
    traces_lists.append(traces)

In [ ]:
traces_lists = pickle.load(open(modules.secret.PACKAGE_PATH + "traces_lists.pkl", "rb"))

In [ ]:
fig = flt_pass.create_admet_progression_figure(
    traces_lists, v_space=0.1, h_space=0.08, y_max=1.3
)
graph = Graph()
graph.update_parameters(dict(width=1000, height=700, annotation_size=28))
graph.style_figure(fig)
fig.update_layout(
    showlegend=True,
    legend=dict(
        x=1.0,
        y=0.5,
        font=dict(size=18),
    ),
)
graph.save_figure(figure=fig, path=EXPORT_PATH, fname=f"{prefix}_{filters}_{target}")
fig.show()

In [ ]:
# fnames = flt_pass.prepare_generation_fnames(
#     prefix="model2_hnh",
#     n_iters=5,
#     channel="admetfg_softsub",
#     filters="ADMET+FGs",
#     target="HNH",
# )
# pp.pprint(fnames)